# Notebook `05_app.ipynb`

---

# 🖥️ Déploiement de l'application interactive

Ce notebook final permet de **mettre en application les modèles de prédiction développés** dans les notebooks précédents.  
Il propose une **interface interactive Gradio** qui permet d'estimer :
- le **prix probable** d’un service Fiverr,
- ainsi que sa **tranche de prix** (Basse, Moyenne ou Haute).

Cette application s’appuie sur un **pipeline hybride** combinant embeddings de texte et variables numériques pour fournir des prédictions accessibles et rapides.

## 🎯 Objectifs

- Intégrer les **fonctions de prédiction** basées sur les modèles enregistrés (`predict_price`, `predict_tranche`)
- Concevoir une **interface utilisateur intuitive** avec `gradio.Blocks`
- Ajouter une fonction de **log** permettant d’enregistrer les prédictions pour analyse
- Proposer une interface flexible :
  - Sélection de la fiabilité par niveau prédéfini (`Acceptable`, `Bonne`, etc.)
  - Ou réglage manuel via un `slider` numérique
- Déployer l'application en local ou sur un serveur

## 🧠 Fonctionnalités clés

- **Entrée utilisateur** :
  - Titre du service (texte libre)
  - Niveau du vendeur (option désactivée ici)
  - Fiabilité (par choix prédéfini ou valeur personnalisée)

- **Prétraitement transparent** :
  - Embedding de la description via `sentence-transformers`
  - Normalisation de la fiabilité
  - Alignement des colonnes avec les modèles entraînés

- **Sorties affichées** :
  - Prédiction du **prix** (`predict_price`)
  - Prédiction de la **tranche de prix** (`predict_tranche`)

- **Fonction de signalement** :
  - Ajout des prédictions dans un fichier `log.csv` pour audit ou analyse ultérieure

## Compétences mobilisées

- **Bloc 3 — C2** : Appliquer un pipeline de transformation cohérent entre entraînement et usage réel.
- **Bloc 5 — C4** : Déployer une application d’IA accessible et interactive (Gradio).

*Ce notebook constitue la version déployable de l’outil de prédiction, à présenter au jury ou à tester par les utilisateurs finaux.*

---

## 🧭 Sommaire

1. [Importation des bibliothèques](#-1-importation-des-bibliothèques)
2. [Préparation du fichier de log](#-2-préparation-du-fichier-de-log)
3. [Définition des niveaux de fiabilité prédéfinis](#-3-définition-des-niveaux-de-fiabilité-prédéfinis)
4. [Fonction de prédiction principale](#-4-fonction-de-prédiction-principale)
5. [Fonction d’enregistrement des prédictions dans le log](#-5-fonction-denregistrement-des-prédictions-dans-le-log)
6. [Interface utilisateur Gradio](#-6-interface-utilisateur-gradio)
7. [Analyse de sensibilité à la fiabilité](#-7-analyse-de-sensibilité-à-la-fiabilité)
8. [Analyse exploratoire : corrélation Fiabilité / Prix](#-8-analyse-exploratoire--corrélation-fiabilité--prix)

---

## 🗃️ 2. Définition des constantes et du modèle d'embedding

### 🧩 2.1. Pourquoi cette étape ?

Avant de mettre en place la logique d’inférence, nous définissons plusieurs **paramètres globaux** :

- Le chemin du fichier de log
- L’URL d’appel de l’API REST (si FastAPI est utilisé)
- Les valeurs de fiabilité prédéfinies, plus simples à sélectionner pour l’utilisateur
- La liste des niveaux de vendeur, utile pour le deep learning

Nous chargeons également dès cette étape le **modèle d’embedding** SentenceTransformer, qui convertira les descriptions en vecteurs numériques dans les cas ML et DL.

### 📌 2.2. Détail des éléments définis

| Élément | Description |
|--------|-------------|
| `LOG_PATH` | Chemin du fichier où seront enregistrées les prédictions signalées |
| `API_URL`  | URL du point de terminaison de l’API FastAPI |
| `choices`  | Dictionnaire de correspondance entre labels de fiabilité et pourcentages |
| `niveau_mapping` | Liste des niveaux de vendeurs utilisés dans le modèle DL |
| `embedding_model` | Modèle SentenceTransformer chargé une seule fois pour éviter les rechargements |

---

### 🐍 2.3. Script de définition

In [39]:
import gradio as gr
import pandas as pd
import numpy as np
import os
import joblib
import requests
import sys
from tensorflow.keras.models import load_model
from sentence_transformers import SentenceTransformer

sys.path.append('../code')  # si on est dans notebooks/
from ml_predict import predict_price, predict_tranche

---

## 🌐 2. Initialisation des chemins API et fichiers

### 🔌 2.1. Pourquoi cette étape ?

Avant toute interaction avec l’API ou enregistrement dans un fichier, il est indispensable de **définir les chemins de travail** :
- l’adresse de l’**API REST** déployée localement (ou à distance),
- le chemin du **fichier de log** dans lequel seront stockées les prédictions signalées.

Ces variables serviront tout au long de l'application, aussi bien pour effectuer les requêtes que pour consigner les résultats.

### 🧭 2.2. Résultat attendu

- Une variable `API_URL` contenant l’adresse de l’API de prédiction.
- Une variable `LOG_PATH` pointant vers le fichier CSV de log (`flagged/log.csv`).

---

### 🐍 2.3. Script de configuration initiale

Le script ci-dessous initialise :
- `LOG_PATH` : le chemin relatif vers le fichier de journalisation des prédictions signalées,
- `API_URL` : l’URL de l’API FastAPI utilisée pour interroger le modèle de prédiction.

In [40]:
# Chemin d’enregistrement des prédictions signalées
LOG_PATH = "flagged/log.csv"

API_DL_URL = "http://127.0.0.1:8000/predict"
API_ML_URL_PRICE = "http://127.0.0.1:8001/predict_price"
API_ML_URL_TRANCHE = "http://127.0.0.1:8001/predict_tranche"

---

## 🎚️ 3. Définition des niveaux de fiabilité prédéfinis

### 🧮 3.1. Pourquoi proposer des niveaux ?

L'utilisateur peut indiquer la **fiabilité du vendeur** soit :
- en **valeur libre** via un curseur (`Slider`),
- soit en sélectionnant un **niveau prédéfini**, plus intuitif et guidé.

Ces niveaux permettent :
- une **expérience utilisateur simplifiée**,
- une **standardisation** des entrées dans le cadre de tests ou d’évaluations.

### 📊 3.2. Correspondance des niveaux

| Niveau choisi    | Valeur numérique (%) |
|------------------|----------------------|
| Acceptable       | 80                   |
| Moyenne          | 85                   |
| Bonne            | 90                   |
| Très Bonne       | 96                   |
| Excellente       | 99                   |

---

### 🐍 3.3. Script de définition des niveaux de fiabilité

Le dictionnaire `choices` associe des niveaux de fiabilité verbaux à des valeurs numériques.  
Cela permet une conversion directe entre une sélection utilisateur et un pourcentage exploitable par le modèle.

In [41]:
# Dictionnaire des niveaux de fiabilité prédéfinis
choices = {
    "Acceptable": 80,     # Fiabilité de base, suffisante pour un débutant ou un profil récent
    "Moyenne": 85,        # Légèrement supérieure à la moyenne, indique un certain sérieux
    "Bonne": 90,          # Bon niveau de confiance dans les avis et les évaluations
    "Très Bonne": 96,     # Profil très fiable, prestations régulièrement bien notées
    "Excellente": 99      # Niveau maximal de fiabilité, presque sans reproche
}

niveau_mapping = {
    "Débutant": 1,
    "Intermédiaire": 2,
    "Expert": 3
}

---

## 🧠 4. Initialisation du modèle de deep learning

### 🤔 4.1. Pourquoi ce modèle ?

Dans le cadre de la soutenance, il est nécessaire de démontrer l'utilisation d'un **modèle de deep learning** (compétence C3).  
Ce modèle prend en entrée :
- des **embeddings** issus de descriptions textuelles,
- ainsi que des variables numériques standardisées (fiabilité, niveau du vendeur...).

Il est entraîné séparément puis intégré dans l’application via un **chargement différé** pour optimiser les ressources.

### 🧾 4.2. Informations chargées

- `niveau_mapping` : liste des niveaux normalisés pour le modèle (`Beginner`, `Intermediate`, `Expert`).
- `embedding_model` : modèle `all-MiniLM-L6-v2` de SentenceTransformers pour convertir les descriptions en vecteurs.
- `deep_model` et `scaler_dl` : initialisés à `None` pour permettre un **lazy loading** (chargement à la demande).

---

### 🐍 4.3. Script d’initialisation

Ce script prépare les éléments nécessaires à l’exécution locale du modèle de deep learning :  
- niveaux de vendeur standardisés,
- transformation de texte en vecteurs,
- et initialisation différée du modèle et de son préprocesseur.

In [42]:
# Liste des niveaux de vendeur (utilisée pour la prédiction deep learning locale)
niveau_mapping = {"Débutant": 1, "Confirmé": 2, "Expert": 3}
# Cette liste correspond aux niveaux de vendeur standardisés pour le modèle de deep learning local.
# Elle est utilisée notamment pour encoder ou décoder les niveaux dans l'ordre prévu par le modèle.

# Chargement différé (lazy loading) du modèle d'embedding de texte
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# Modèle de transformation des descriptions textuelles en vecteurs numériques (384 dimensions).
# Utilisé pour alimenter le modèle deep learning.

# Initialisation différée du modèle deep learning et de son scaler
deep_model = None   # Le modèle sera chargé ou défini dynamiquement lors du premier appel de prédiction.
scaler_dl = None    # Le scaler associé au modèle DL (normalisation des données d’entrée) sera également chargé à la volée.

---

## 🗂️ 5. Chargement du modèle deep learning et du scaler

### 🔄 5.1. Pourquoi un chargement différé ?

Le modèle de deep learning et son scaler ne sont pas chargés dès le démarrage pour **économiser la mémoire**.  
Ils sont initialisés **à la demande** grâce à une fonction dédiée, ce qui permet :
- d’éviter de ralentir le démarrage de l’application,
- de limiter l’utilisation des ressources tant qu’une prédiction DL n’est pas requise.

### 📦 5.2. Ressources chargées

- `deep_model.h5` : modèle entraîné avec **Keras/TensorFlow**, stocké dans le dossier `models/deep/`.
- `scaler.pkl` : objet `StandardScaler` (ou similaire) utilisé pour **normaliser les variables numériques**.

---

### 🐍 5.3. Script de chargement conditionnel

La fonction `load_deep_models()` utilise des variables globales pour ne charger les modèles qu’une seule fois.  
Elle garantit ainsi que le modèle et le scaler sont prêts **uniquement quand nécessaire**, sans rechargement inutile.

In [43]:
def load_deep_models():
    """
    Charge les modèles nécessaires au deep learning si ce n’est pas déjà fait.
    Utilise le mot-clé 'global' pour modifier les variables définies à l'extérieur de la fonction.
    """
    global deep_model, scaler_dl  # On déclare vouloir utiliser/modifier les variables globales

    if deep_model is None:
        # Chargement du modèle de deep learning (réseau de neurones)
        deep_model = load_model("../models/deep/deep_model_notebook.h5")
        # Ce fichier contient le modèle entraîné avec Keras (format HDF5)

        # Chargement du scaler pour normaliser les variables numériques
        scaler_dl = joblib.load("../models/deep/scaler_notebook.pkl")
        # Le scaler assure la cohérence entre les données d'entraînement et les données de prédiction

---

## 🔮 6. Fonction de prédiction principale

### 🧠 6.1. Objectif

Cette fonction est le **cœur du système de prédiction**.  
Elle combine :
- le choix de la **source de fiabilité** (niveau prédéfini ou valeur libre),
- la **prédiction du prix exact** (modèle de régression),
- la **prédiction de la tranche de prix** (modèle de classification).

Elle assure également la **gestion des erreurs** en cas de mauvaise saisie ou de problème technique.

### ⚙️ 6.2. Paramètres

| Paramètre           | Type     | Description                                                                 |
|---------------------|----------|-----------------------------------------------------------------------------|
| `description`        | `str`    | Texte décrivant le service Fiverr                                          |
| `niveau`             | `str`    | Niveau du vendeur (`Beginner`, `Intermediate`, `Expert`)                  |
| `use_predefined`     | `bool`   | Indique si on utilise un niveau de fiabilité prédéfini                     |
| `fiabilite_percent`  | `float`  | Valeur manuelle de fiabilité (%) si `use_predefined` est `False`          |
| `fiabilite_choix`    | `str`    | Nom du niveau de fiabilité choisi si `use_predefined` est `True`           |
| `modele`             | `str`    | Méthode à utiliser : `ML - Local`, `DL - Local`, ou `API - FastAPI`        |

### 🎯 6.3. Résultat

Retourne deux prédictions :
- le **prix estimé** (float, arrondi à 2 décimales),
- la **tranche de prix** (`Basse`, `Moyenne`, `Haute` ou `"Non évaluée"` selon le modèle).

En cas d'erreur, un message d'alerte est retourné à la place du prix.

---

### 🐍 6.4. Script de prédiction principale

Le script détecte automatiquement le type de modèle à utiliser :
- local ML avec classification (`predict_price`, `predict_tranche`),
- deep learning local (embeddings + scaler + `deep_model`),
- ou appel distant à une API REST (`/predict`).

Chaque bloc est encapsulé dans une structure `try/except` pour éviter les plantages en cas d'erreur réseau, de chargement ou de traitement.

In [44]:
# Fonction principale de prédiction
def faire_une_prediction(description, niveau_label, use_predefined, fiabilite_percent, fiabilite_choix, modele):
    """
    Fonction centrale permettant de prédire un prix (et une tranche si applicable) 
    selon le modèle sélectionné : local ML, local DL ou API distante.
    """

    # Détermination de la fiabilité (soit via choix prédéfini, soit via valeur libre)
    fiabilite = (choices[fiabilite_choix] if use_predefined else fiabilite_percent) / 100
    niveau = niveau_mapping.get(niveau_label, 1)

    try:
        # Modèle machine learning local (ex. XGBoost + classification)
        if modele == "ML - Local":

            prix = predict_price(description, fiabilite) * 10
            tranche = predict_tranche(description, fiabilite)

        # Modèle deep learning local (Keras + embeddings)
        elif modele == "DL - Local":

            load_deep_models()  # Chargement du modèle et du scaler
            
            emb = embedding_model.encode([description]).flatten() # Embedding de la description textuelle

            niveau_ohe = [1 if niveau == n else 0 for n in [1, 2, 3]] # One-hot encoding du niveau vendeur
            
            features = np.hstack([emb, niveau_ohe, [fiabilite]]) # Assemblage des features : [embeddings] + [niveau OHE] + [fiabilité]
            
            features_scaled = scaler_dl.transform([features]) # Mise à l’échelle des features
            
            prix = deep_model.predict(features_scaled)[0][0] * 10 # Prédiction avec le modèle DL
            
            tranche = "Non évaluée"  # Le modèle DL ne fournit pas encore de classification

        # Prédiction via DL FastAPI
        elif modele == "DL - FastAPI":

            response = requests.post(
                API_DL_URL,
                json={"description": description, "niveau": niveau, "fiabilite": fiabilite},
                timeout=5
            )
            response.raise_for_status()
            data = response.json()
            prix = data.get("prix", -1) * 10
            tranche = data.get("tranche", "Non évaluée")
            
        # Prédiction via ML FastAPI
        elif modele == "ML - FastAPI":

            prix = -1
            tranche = "Non évaluée"

            # Appel API pour le prix
            response_price = requests.post(
                API_ML_URL_PRICE,
                json={"description": description, "fiabilite": fiabilite},
                timeout=5
            )
            response_price.raise_for_status()
            prix = response_price.json().get("prix", -1) * 10

            # Appel API pour la tranche
            response_tranche = requests.post(
                API_ML_URL_TRANCHE,
                json={"description": description, "fiabilite": fiabilite},
                timeout=5
            )
            response_tranche.raise_for_status()
            tranche = response_tranche.json().get("tranche", "Non évaluée")

        # Cas de modèle inconnu
        else:
            return "Modèle inconnu", ""

        # Résultat final : prix arrondi et tranche
        return round(prix, 2), tranche

    except Exception as e:
        # Gestion des erreurs inattendues
        return f"Erreur : {str(e)}", ""


---

## 📝 7. Fonction d’enregistrement des prédictions dans le log

### 🧠 7.1. Objectif

Cette fonction permet de **conserver un historique des prédictions** réalisées via l’application Gradio.  
Elle enregistre chaque estimation dans un fichier `log.csv`, avec les données d’entrée et les résultats associés.

### 🗃️ 7.2. Données enregistrées

| Colonne            | Description                                             |
|--------------------|---------------------------------------------------------|
| `Description`       | Texte fourni par l’utilisateur                          |
| `Fiabilité (%)`     | Valeur de fiabilité (selon sélection manuelle ou prédéfinie) |
| `Prix prédit (€)`   | Résultat de la prédiction de prix (régression)         |
| `Tranche prédite`   | Résultat de la classification (`Basse`, `Moyenne`, `Haute`) |
| `Modèle utilisé`    | Type de modèle utilisé (`ML - Local`, `DL - Local`, `API`)   |

### 🧾 7.3. Comportement

- Le fichier `log.csv` est **créé s’il n’existe pas**, ou mis à jour en **mode ajout** (`append`).
- Chaque prédiction est **enregistrée sur une ligne distincte**.

> ℹ️ Ce fichier log peut servir de **trace d’utilisation**, ou de base pour des analyses a posteriori.

---

### 🐍 7.4. Script d’enregistrement dans le fichier log

Le script ci-dessus construit dynamiquement une ligne de données à partir des entrées utilisateur et du résultat de prédiction.  
Il gère automatiquement la création ou la mise à jour du fichier `log.csv` situé dans le répertoire `flagged/`.

In [45]:
# Fonction d'enregistrement d'une prédiction dans le fichier log
def enregistrer_log(description, use_predefined, fiabilite_percent, fiabilite_choix, prix, tranche, modele):
    """
    Enregistre une prédiction dans un fichier CSV de log.
    Gère le formatage, la sélection de la fiabilité et la création/extension du fichier.
    """

    # Détermination de la fiabilité selon le mode sélectionné
    fiabilite = choices[fiabilite_choix] if use_predefined else fiabilite_percent

    # Construction du dictionnaire de données à enregistrer
    log_data = {
        "Description": description,
        "Fiabilité (%)": fiabilite,
        "Prix prédit (€)": prix,
        "Tranche prédite": tranche,
        "Modèle utilisé": modele
    }

    # Conversion en DataFrame (une seule ligne)
    df_log = pd.DataFrame([log_data])
    os.makedirs(os.path.dirname(LOG_PATH), exist_ok=True)

    # Ajout au fichier existant ou création si nécessaire
    if os.path.exists(LOG_PATH):
        # Le fichier existe déjà : ajout en mode append sans les en-têtes
        df_log.to_csv(LOG_PATH, mode="a", index=False, header=False)
    else:
        # Le fichier n’existe pas encore : création avec en-têtes
        df_log.to_csv(LOG_PATH, index=False)

    # Message de retour pour l’utilisateur
    return "Signalement enregistré avec succès."

---

## 🖥️ 8. Interface utilisateur Gradio

### 🎯 8.1. Objectif

Proposer une **interface interactive et intuitive** pour permettre à l’utilisateur :
- de saisir les informations d’un service Fiverr,
- d’obtenir immédiatement une **estimation du prix** (`Régression`),
- et sa **tranche de prix** (`Classification`),
- puis de **signaler** le cas au fichier `log.csv` si besoin.

### 🧰 8.2. Composants principaux

| Élément                    | Fonction                                                                 |
|----------------------------|--------------------------------------------------------------------------|
| `Textbox` (description)    | Entrée textuelle décrivant le service                                   |
| `Dropdown` (niveau)        | Choix du niveau du vendeur (actuellement masqué, future évolution)      |
| `Checkbox` / `Radio`       | Sélection du niveau de fiabilité (manuelle ou prédéfinie)               |
| `Button`                   | Lance la prédiction (`📈 Estimer le prix`)                              |
| `Textbox` (sorties)        | Affichage des résultats (`Prix estimé`, `Tranche estimée`)              |
| `Button`                   | Enregistrement de la prédiction dans le fichier log (`🚨`)              |
| `Textbox` (confirmation)   | Message de confirmation en cas de signalement réussi                    |

### 🔁 8.3. Logique d’interaction

- **Fiabilité personnalisée ou prédéfinie** :  
  L’utilisateur peut choisir d’utiliser un curseur libre (pourcentage) ou un niveau de fiabilité prédéfini.

- **Synchronisation dynamique** :  
  Le radio bouton met à jour la valeur du slider et inversement si on décoche la case.

- **Traitement centralisé** :  
  Les fonctions `faire_une_prediction` et `enregistrer_log` pilotent la logique métier.

---

### 🐍 8.4. Script Gradio (création de l’interface)

Le script ci-dessus crée une interface structurée en deux colonnes :
- à gauche : les **paramètres d’entrée** utilisateur,
- à droite : les **résultats** de prédiction et les **actions de signalement**.

L’application est directement **lancée avec `iface.launch()`**.

In [46]:
# Interface utilisateur Gradio
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=1):
            description = gr.Textbox(label="Description du service", value="Je développe un site web", lines=2, placeholder="Je fais des audits qualité...")
            use_predefined = gr.Checkbox(label="Utiliser un niveau de fiabilité prédéfini", value=True, visible=False)
            fiabilite_percent = gr.Slider(label="Fiabilité (%)", minimum=0, maximum=100, value=80, step=5, visible=False)
            fiabilite_choix = gr.Radio(label="Niveau de fiabilité", choices=list(choices.keys()), value="Acceptable", visible=True)
            modele = gr.Radio(
                label="Modèle à utiliser",
                choices=["ML - Local", "ML - FastAPI", "DL - Local", "DL - FastAPI"],
                value="ML - Local"
            )
            niveau = gr.Dropdown(label="Niveau du vendeur", choices=list(niveau_mapping.keys()), value="Débutant", visible=False)
            bouton_predire = gr.Button("Estimer le prix")

        with gr.Column(scale=1):
            sortie_prix = gr.Textbox(label="Prix estimé en €/h",  value="Attente de prédiction", visible=False)
            sortie_tranche = gr.Textbox(label="Tendance des prix sur le marché",  value="Attente de prédiction", visible=True)
            bouton_signaler = gr.Button("Ajouter au fichier log.csv")
            confirmation = gr.Textbox(label="Confirmation", visible=False)

    # === Fonctions d'interactions ===

    def sync_slider_with_radio(choix):
        return gr.update(value=choices[choix])
    
    fiabilite_choix.change(sync_slider_with_radio, inputs=fiabilite_choix, outputs=fiabilite_percent)

    def toggle_inputs(use_predef):
        return {
            fiabilite_percent: gr.update(visible=not use_predef),
            fiabilite_choix: gr.update(visible=use_predef)
        }

    use_predefined.change(toggle_inputs, inputs=use_predefined, outputs=[fiabilite_percent, fiabilite_choix])

    def toggle_niveau(model_choice):
        return gr.update(visible=model_choice in ["DL - Local", "DL - FastAPI"])

    modele.change(toggle_niveau, inputs=modele, outputs=niveau)

    def toggle_prix_visibility(model_choice):
        return gr.update(visible=model_choice in ["DL - Local", "DL - FastAPI"], value="Attente de prédiction")

    modele.change(toggle_prix_visibility, inputs=modele, outputs=sortie_prix)

    def toggle_tranche_visibility(model_choice):
        return gr.update(visible=model_choice in ["ML - Local", "ML - FastAPI"], value="Attente de prédiction")

    modele.change(toggle_tranche_visibility, inputs=modele, outputs=sortie_tranche)

    def reset_sortie_prix(model_choice):
        return gr.update(value="Attente de prédiction")

    modele.change(reset_sortie_prix, inputs=modele, outputs=sortie_prix)

    def faire_une_prediction(description, niveau, use_predef, fiab_percent, fiab_choix, model):
        # À remplacer par ta logique réelle de prédiction
        return "12.5", "Moyenne"

    def enregistrer_log(description, use_predef, fiab_percent, fiab_choix, prix, tranche, model):
        # À remplacer par ta logique réelle d’enregistrement
        return "✅ Prédiction enregistrée dans le fichier log.csv"

    bouton_predire.click(
        fn=faire_une_prediction,
        inputs=[description, niveau, use_predefined, fiabilite_percent, fiabilite_choix, modele],
        outputs=[sortie_prix, sortie_tranche]
    )

    bouton_signaler.click(
        fn=enregistrer_log,
        inputs=[description, use_predefined, fiabilite_percent, fiabilite_choix, sortie_prix, sortie_tranche, modele],
        outputs=confirmation
    )

demo.launch()

#iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.39.0, however version 4.44.1 is available, please upgrade.
--------


## 🧠 Justification des incohérences apparentes dans les tranches prédictives

### ❓ Problème observé

Dans certains cas, il peut sembler incohérent qu’une **tranche de prix estimée** soit plus élevée pour une **fiabilité "Acceptable"** que pour une fiabilité "Excellente". Par exemple :

- Pour une **fiabilité Acceptable (80%)**, la tranche prédite est **Haute**
- Pour une **fiabilité Excellente (99%)**, la tranche prédite est **Basse**

### 🧩 Explication technique

Cette situation peut être expliquée de façon cohérente par la conception du pipeline :

1. **Deux modèles indépendants sont utilisés** :
   - Un modèle de **régression** qui prédit un **prix numérique**
   - Un modèle de **classification** qui prédit une **tranche catégorielle** (`Basse`, `Moyenne`, `Haute`)
   - Ces deux modèles sont **entraînés séparément**, sur des cibles différentes

2. **La classification ne dépend pas du prix prédit** :
   - Le modèle de classification apprend à prédire une **catégorie** (la tranche réelle observée) en fonction d’un ensemble de variables, notamment :
     - L’embedding de la description
     - La fiabilité pondérée
   - Il ne connaît **ni le prix réel, ni le prix prédit par le modèle de régression**

3. **La fiabilité pondérée n’a pas une influence linéaire** :
   - Une meilleure fiabilité (Excellente) ne garantit pas forcément un prix plus élevé ni une tranche plus haute
   - En pratique, **le modèle peut estimer qu’un service générique très fiable** (par exemple un intitulé trop large ou basique) appartient aux tranches basses
   - Tandis qu’un **service plus ciblé ou spécialisé** avec une fiabilité moyenne peut être associé aux tranches hautes

### 📌 Interprétation métier

Ce comportement n'est pas une erreur mais une **manifestation de la logique d'apprentissage supervisé** :

> Le modèle de classification apprend des **règles implicites** présentes dans les données d’entraînement. Il peut considérer certaines descriptions comme typiquement premium, indépendamment du niveau de fiabilité, ce qui entraîne une tranche "Haute" même avec une fiabilité modeste.

### ✅ Conclusion

Il est important de retenir :

- Le système n’utilise **pas une simple règle à seuils** sur le prix pour classer les tranches
- Il s’agit d’un modèle **entraîné à partir d’observations réelles**
- Les incohérences apparentes sont souvent dues à des **corrélations non triviales** dans les données (nature du service, vocabulaire utilisé, etc.)

Cette explication souligne la complexité du comportement des modèles d’IA, et la nécessité de les évaluer **sur l’ensemble du pipeline**, pas uniquement sur un cas particulier.

---

## 🔍 9. Analyse de sensibilité à la fiabilité

### 🧪 9.1. Objectif de cette cellule

Vérifier que la **variable `Fiabilité` a bien un impact** significatif sur le prix prédit.  
On fait varier la valeur de fiabilité de 0 % à 100 %, tout en gardant constante la description du service.

### 📊 9.2. Méthodologie

- Le service testé est simplement : `"Je fais le ménage"`
- On fait évoluer la fiabilité de 0.00 à 1.00 (i.e. de 0 % à 100 %)
- Pour chaque niveau de fiabilité, on **prévoit le prix** avec le modèle de régression finalisé
- On affiche ensuite les résultats dans la console

### 🔍 9.3. Ce que l'on cherche à observer

- Une **augmentation globale du prix** avec la fiabilité croissante (logique métier)
- Une **zone d'inflexion ou palier éventuel** aux alentours de 0.8 (80 %) — critère de haute fiabilité
- Des **résultats cohérents** avec ceux observés dans l’application Gradio

> 💡 *Cette étape permet de valider qualitativement la sensibilité du modèle aux variations d’un indicateur clé métier.*

---

### 🐍 9.4. Script de test d’impact de la fiabilité

In [47]:
# Exemple de description et niveau à tester
desc = "Je fais le ménage"

# Test de l’impact de la fiabilité sur la prédiction du prix
for f in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]:
    prix = predict_price(desc, f)
    print(f"Fiabilité = {f*100:.0f}% → Prix prédit = {prix} €")

Fiabilité = 0% → Prix prédit = 1.38 €
Fiabilité = 10% → Prix prédit = 1.38 €
Fiabilité = 20% → Prix prédit = 1.38 €
Fiabilité = 30% → Prix prédit = 1.38 €
Fiabilité = 40% → Prix prédit = 1.38 €
Fiabilité = 50% → Prix prédit = 1.38 €
Fiabilité = 60% → Prix prédit = 1.38 €
Fiabilité = 70% → Prix prédit = 1.38 €
Fiabilité = 80% → Prix prédit = 1.38 €
Fiabilité = 81% → Prix prédit = 1.38 €
Fiabilité = 82% → Prix prédit = 1.38 €
Fiabilité = 83% → Prix prédit = 1.38 €
Fiabilité = 84% → Prix prédit = 1.42 €
Fiabilité = 85% → Prix prédit = 1.42 €
Fiabilité = 90% → Prix prédit = 1.49 €
Fiabilité = 91% → Prix prédit = 1.49 €
Fiabilité = 92% → Prix prédit = 1.49 €
Fiabilité = 93% → Prix prédit = 1.49 €
Fiabilité = 94% → Prix prédit = 1.54 €
Fiabilité = 95% → Prix prédit = 1.59 €
Fiabilité = 96% → Prix prédit = 1.61 €
Fiabilité = 97% → Prix prédit = 1.68 €
Fiabilité = 98% → Prix prédit = 1.73 €
Fiabilité = 99% → Prix prédit = 1.73 €
Fiabilité = 100% → Prix prédit = 1.72 €


---

## 📊 10. Analyse exploratoire : corrélation Fiabilité / Prix

### 🧪 10.1. Objectif de cette cellule

Vérifier empiriquement si la **fiabilité du vendeur est corrélée au prix proposé**.  
Cela permet de **valider la pertinence** de cette variable comme feature dans notre modèle.

### 🧮 10.2. Statistiques de la variable `Fiabilite`

- Affichage de la distribution statistique (`min`, `max`, `mean`, `std`, etc.)
- Vérification que la variable **varie suffisamment** pour avoir un potentiel explicatif

### 🔗 10.3. Corrélation entre `Fiabilite` et `Prix`

- Calcul du **coefficient de corrélation de Pearson**
- Permet de détecter une **relation linéaire potentielle** entre la variable explicative et la cible

> 💡 Une corrélation significative (positive ou négative) soutient l'idée que cette variable a un **rôle déterminant** dans le modèle de prédiction.

---

### 🐍 10.4. Script d’analyse exploratoire

In [48]:
df = pd.read_csv("../data/fiverr_cleaned_dl_notebook.csv")
print(df["Fiabilite"].describe())
print(df[["Fiabilite", "Prix"]].corr())

count    1145.000000
mean        1.116113
std         0.530929
min         0.081877
25%         0.714403
50%         0.811702
75%         1.653477
max         2.620674
Name: Fiabilite, dtype: float64
           Fiabilite      Prix
Fiabilite   1.000000  0.125243
Prix        0.125243  1.000000
